## 六步搭建网络

- **import** 导入相关模块

- **train,test** 划分训练数据集和测试数据集

- **model = keras.models.Sequential** 搭建网络结构，相当于走了一边前向传播

- **model.compile** 配置训练时的优化器、损失函数、评测指标

- **model.fit** 训练网络，配置训练集和测试集的特征、迭代次数、每次喂入的大小

- **model.summary** 查看训练后的网络结构

## Sequential([网络结构])

例如：

拉直层：`keras.layers.Flatten()`，不做计算，只将输入特征拉直成一维

全连接层：`keras.layers.Dense(神经元个数, activation='激活函数', kernel_regularizer=正则化)`

> `activation`使用字符串配置，可选用的有：relu、softmax、sigmoid、tanh

> `kernel_regularizer`可选的有：keras.regularizers.l1()、keras.regularizers.l2()

卷积层： `keras.layers.Conv2D(filters=卷积核个数, kernel_size=卷积核尺寸, strides=卷积步长, padding='vaild'or'same')`

循环神经网络(LSTM)层：`keras.layers.LSTM()`

## model.compile(optimizer=优化器, loss=损失函数, metrics=['准确率'])

### optimizer（优化器）

`'sgd'`or`keras.optimizer.SGD(lr=学习率, momentum=动量参数)`

`'adagrad'`or`keras.optimizer.Adagrad(lr=学习率)`

`'adadelta'`or`keras.optimizer.Adadelta(lr=学习率)`

`'adam'`or`keras.optimizer.Adam(lr=学习率, beta_1=0.9, beta_2=0.999)`

### loss（损失函数）

`'mse'`or`keras.losses.MeanSquaredError()`

`'sparse_categorical_crossentropy'`or`keras.losses.SparesCategoricalCrossentropy(form_logits=False)`

> `form_logits`是否是原始输出，即就是没有经过概率分布的输出，若神经网络输出前需要经过概率分布处理则`form_logits=False`

### metrics（评测指标）

`accuracy`：y_和y都是数值形式，例如：y_=[1] y= [1]，也就是分类标签用数值表示，输出结果也是数值

`categorical_accuracy`：y_和y都是独热码(概率分布)，例如：y_=[1,0,0] y=[0.33, 0.43, 0.32]

`sparese_categorical_accuracy`：y_是数值，y是独热码表示，例如：y_=[1] y=[0.256,0.695, 0.048]

## model.fit(训练集的输入特征, 训练集的标签,  batch_size=每次喂入的大小, epochs=循环多少轮, validation_data=（测试集的输入特征，测试集的标签）, validation_split=从训练集划分出多少比例的测试集, validation_freq=多少次epoch回测一次)

## 数据增强（增大数据量）

```py
image_gen_train = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 所有数据将要与该数值相乘
    rotation_range= 随机旋转角度数范围
    width_shift_range= 随机宽度偏移量
    height_shift_range= 随机高度偏移量
    horizontal_flip= 是否水平反转
    zoom_range= 随机缩放范围[1-n, 1+n]
)

image_gen_train.fit(x_train)
```

例如：
```py 
image_gen_train = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255., #将图像中的元素全部除以255
    rotation_range=45, #随机45度旋转
    width_shift_range=.15, #宽度偏移0.15倍
    height_shift_range=.15, #高度偏移0.15倍
    horizontal_flip=False, #不进行水平翻转
    zoom_range=.5 #将图像随机缩放.5倍 
)
```

`image_gen_train.fit(x_train)`需要输入的数据是4维数据即(个数，宽度， 高度， 通道数)

```py 
#将x_train 从(60000, 28, 28)变形为(60000, 28, 28, 1)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
```

## 断点续训

### 读取模型
```py 
model.load_weights(模型保存路径)
```
例如：
```py 
checkpoint_save_path = './checkpoint/mnist.ckpt'
# 因为模型保存为ckpt文件时，会同步生成索引表.index,所以通过判断是否又索引表就可以判断是否有保存好的模型
if os.path.exists(checkpoint_save_path + '.index'):
    print('------load the model-------')
    model.load_weights(checkpoint_save_path)
```

### 保存模型
```py
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=模型保存路径, #文件存储路径
    save_weights_only=True/False, #是否只保留模型参数
    save_best_only=True/False, #是否只保留最优结果
)

#执行训练过程时
history = model.fit(callback=[cp_callback])
```

例如：
```py 
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_save_path,
    save_weights_only=True,
    save_best_only=True
)

history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1, callbacks=[cp_callback])
```

## 参数提取

### 提取可选参数
`model.trainable_variables`返回模型中可训练的参数

### 设置print输出格式

`np.set_printoptions(threshold=超过多少省略显示)`

例如:

np.inf表示无限大
`np.set_printoptions(threshold=np.inf)`

### 将训练参数存入文本

```py 
print(model.trainable_variables)
file = open('weights.txt', 'w')
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()
```

## 可视化acc曲线和loss曲线

```py 
history = model.fit(训练数据集，训练标签集，batch_size=，epochs=，
    validation_split=用作测试的数据比例，
    validation_data=测试集，#只能和validation_split存在一个
    validation_freq=测试间隔 #即训练多少轮测试一次
)
```

history:

- 训练集loss: loss
- 测试集loss: val_loss
- 训练集准确率: sparse_categorical_accuracy
- 测试集准确率: val_sparse_categorical_accuracy

代码：

```py 
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Train and Validation Loss')
plt.legend()
plt.show()
```